In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Define the neural network
class SinApproximationNet(nn.Module):
    def __init__(self):
        super(SinApproximationNet, self).__init__()
        self.fc1 = nn.Linear(25, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 14)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# Generate the dataset
def generate_dataset(num_samples=1000):
    x = torch.linspace(-2 * torch.pi, 2 * torch.pi, num_samples).unsqueeze(1)
    X = torch.cat([x ** i for i in range(25)], dim=1)
    Y = torch.cat([torch.sin((i + 1) * x) for i in range(14)], dim=1)
    return X, Y

In [6]:
# Split the dataset into training and testing sets using DataLoader
def create_data_loaders(X, Y, batch_size=32, test_split=0.2):
    dataset = TensorDataset(X, Y)
    test_size = int(test_split * len(dataset))
    train_size = len(dataset) - test_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

In [7]:
# Training the network
def train_model(model, criterion, optimizer, train_loader, epochs=1000):
    model.train()
    epoch_losses = []
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        epoch_loss /= len(train_loader)
        epoch_losses.append(epoch_loss)
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}')
    
    return epoch_losses

In [8]:
# Testing the network
def test_model(model, test_loader):
    model.eval()
    total_loss = 0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for batch_X, batch_Y in test_loader:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(test_loader)
    print(f'Test Loss: {avg_loss:.4f}')
    return avg_loss

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Instantiate the model, define the loss function and the optimizer
model = SinApproximationNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate the dataset
X, Y = generate_dataset(num_samples=10000)
X, Y = X.to(device), Y.to(device)

Using device: cuda


In [10]:
# Create data loaders
train_loader, test_loader = create_data_loaders(X, Y, batch_size=32, test_split=0.2)

# Train the model
epoch_losses = train_model(model, criterion, optimizer, train_loader, device, epochs=1000)

TypeError: train_model() got multiple values for argument 'epochs'

In [ ]:
# Test the model
test_model(model, test_loader, device)

In [ ]:
# Plot the training loss over epochs
plt.plot(epoch_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.show()

# Example prediction
model.eval()
with torch.no_grad():
    for batch_X, batch_Y in test_loader:
        predicted = model(batch_X)
        break

# Print a few test outputs and corresponding ground truth values for comparison
print("Predicted vs Actual values:")
for i in range(5):
    print(f'Predicted: {predicted[i].numpy()}, Actual: {batch_Y[i].numpy()}')
